# Testing tool calling

In [ ]:
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Raw OpenAI client
client = OpenAI(
    api_key="YOU_API_KEY",
    base_url="https://api.lapathoniia.top"
)

# LangChain tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b

def test_raw_openai():
    print("=== Testing Raw OpenAI Client ===")
    
    tools = [{
        "type": "function",
        "function": {
            "name": "add_numbers",
            "description": "Add two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer"},
                    "b": {"type": "integer"}
                },
                "required": ["a", "b"]
            }
        }
    }]
    
    response = client.chat.completions.create(
        model="lapa-function-calling",
        messages=[{"role": "user", "content": "Add 15 and 27 using the function"}],
        tools=tools,
        tool_choice="required"  # Force function calling
    )
    
    print(f"Response: {response.choices[0].message}")
    if response.choices[0].message.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.choices[0].message.tool_calls:
            print(f"Function: {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
            
            # Execute the function
            import json
            args = json.loads(tool_call.function.arguments)
            result = args['a'] + args['b']
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")

def test_langchain():
    print("\n=== Testing LangChain ===")
    
    llm = ChatOpenAI(
        model="gpt-5.2",
        api_key="",     
        # base_url="https://api.lapathoniia.top"
    )
    
    llm_with_tools = llm.bind_tools([add_numbers])#, tool_choice="any")
    
    response = llm_with_tools.invoke([HumanMessage(content="Use the add_numbers function to add 15 and 27")])
    
    print(f"Tool calls: {response.tool_calls}")
    if response.tool_calls:
        print("✓ Function called successfully!")
        for tool_call in response.tool_calls:
            print(f"Function: {tool_call['name']}")
            print(f"Arguments: {tool_call['args']}")
            
            # Execute the function
            result = add_numbers.invoke(tool_call['args'])
            print(f"Function result: {result}")
    else:
        print("✗ No function calls made")
        print(f"Text response: {response.content}")

In [ ]:
test_langchain()

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-X_GR0JPbgYsIVRspba2ArA",
    base_url="https://api.lapathoniia.top"
)

    
response = client.chat.completions.create(
    model="lapa-function-calling",
    messages=[{"role": "user", "content": "Як тебе звати?"}],
)

print(f"Response: {response.choices[0].message}")

In [3]:
# Compare token counts: Lapa-function-calling vs Gemma-3 tokenizer
from transformers import AutoTokenizer
from huggingface_hub import login
import sys
sys.path.append('..')
from agent.prompts import SYSTEM_PROMPT_UK, SYSTEM_PROMPT_EN

# Login to Hugging Face (required for gated models like Gemma)
login()

# Load tokenizers (use token=True to use the stored HF token)
lapa_tokenizer = AutoTokenizer.from_pretrained("TymofiiNasobko/Lapa-function-calling")
gemma_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-12b-pt", token=True)

# Tokenize both prompts with both tokenizers
lapa_tokens_uk = lapa_tokenizer.encode(SYSTEM_PROMPT_UK)
gemma_tokens_uk = gemma_tokenizer.encode(SYSTEM_PROMPT_UK)
lapa_tokens_en = lapa_tokenizer.encode(SYSTEM_PROMPT_EN)
gemma_tokens_en = gemma_tokenizer.encode(SYSTEM_PROMPT_EN)

print("=" * 70)
print("TOKEN COUNT COMPARISON: Lapa vs Gemma-3 Tokenizer")
print("=" * 70)

print("\n📘 SYSTEM_PROMPT_UK (Ukrainian)")
print("-" * 70)
print(f"Character count: {len(SYSTEM_PROMPT_UK)}")
print(f"Lapa tokenizer:  {len(lapa_tokens_uk)} tokens (avg {len(SYSTEM_PROMPT_UK) / len(lapa_tokens_uk):.2f} chars/token)")
print(f"Gemma tokenizer: {len(gemma_tokens_uk)} tokens (avg {len(SYSTEM_PROMPT_UK) / len(gemma_tokens_uk):.2f} chars/token)")
print(f"Difference: {len(lapa_tokens_uk) - len(gemma_tokens_uk)} tokens ({((len(lapa_tokens_uk) / len(gemma_tokens_uk)) - 1) * 100:+.2f}%)")

print("\n📗 SYSTEM_PROMPT_EN (English)")
print("-" * 70)
print(f"Character count: {len(SYSTEM_PROMPT_EN)}")
print(f"Lapa tokenizer:  {len(lapa_tokens_en)} tokens (avg {len(SYSTEM_PROMPT_EN) / len(lapa_tokens_en):.2f} chars/token)")
print(f"Gemma tokenizer: {len(gemma_tokens_en)} tokens (avg {len(SYSTEM_PROMPT_EN) / len(gemma_tokens_en):.2f} chars/token)")
print(f"Difference: {len(lapa_tokens_en) - len(gemma_tokens_en)} tokens ({((len(lapa_tokens_en) / len(gemma_tokens_en)) - 1) * 100:+.2f}%)")

print("\n📊 SUMMARY")
print("-" * 70)
print(f"UK prompt is {len(lapa_tokens_uk) / len(lapa_tokens_en):.2f}x larger than EN (Lapa tokenizer)")
print(f"UK prompt is {len(gemma_tokens_uk) / len(gemma_tokens_en):.2f}x larger than EN (Gemma tokenizer)")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

TOKEN COUNT COMPARISON: Lapa vs Gemma-3 Tokenizer

📘 SYSTEM_PROMPT_UK (Ukrainian)
----------------------------------------------------------------------
Character count: 8247
Lapa tokenizer:  2341 tokens (avg 3.52 chars/token)
Gemma tokenizer: 2748 tokens (avg 3.00 chars/token)
Difference: -407 tokens (-14.81%)

📗 SYSTEM_PROMPT_EN (English)
----------------------------------------------------------------------
Character count: 3782
Lapa tokenizer:  1014 tokens (avg 3.73 chars/token)
Gemma tokenizer: 1017 tokens (avg 3.72 chars/token)
Difference: -3 tokens (-0.29%)

📊 SUMMARY
----------------------------------------------------------------------
UK prompt is 2.31x larger than EN (Lapa tokenizer)
UK prompt is 2.70x larger than EN (Gemma tokenizer)
